In [1]:
# Test MPC

# Standard imports for SCP via JAX and CVXPY
import numpy as np # CVXPY uses numpy
import cvxpy as cvx # For Convex Optimization
from cvxpy.constraints import constraint
import jax # For computing gradients and JIT
import jax.numpy as jnp
from tqdm import tqdm # For progress bars
from functools import partial # For helping JAX

# Assessment
import time
import matplotlib.pyplot as plt
import scipy.interpolate
import scipy.optimize

import dynamics as dn
import control as ct

## Import NN model

In [2]:
import tensorflow as tf
from tensorflow import keras
model = keras.models.load_model('NN_model_6')


KeyboardInterrupt



## Compute max and min range of dataset for normalization

In [ ]:
import pandas as pd
# load csv
df = pd.read_csv('training_data.csv') #TODO
#df = df.loc[()] #if necessary, conditionally select relevant data
print(df.shape)

mins = np.array(np.min(df,axis=0))
maxs = np.array(np.max(df,axis=0))

norm_inputs = np.vstack([np.hstack([mins[1:9], mins[10:16]]), np.hstack([maxs[1:9], maxs[10:16]])])
norm_outputs = np.vstack([mins[17:], maxs[17:]])

print(norm_inputs)
print(norm_outputs)

## Common Parameters

In [ ]:
import pykep
# SPACECRAFT PARAMETERS
C_D = 1.5 # Dimensionless
C_SRP = 1.5 # Dimensionless

A = 1.0   # m^2
MASS_SAT = 100   # kg

GAMMA_SRP = C_SRP * A / MASS_SAT
GAMMA_DRAG = C_D * A / MASS_SAT

PSI = 0    # thermal oscillation

R_EARTH = 6378
MU_EARTH_KM = pykep.MU_EARTH * 1e-9
EARTH_TO_SUN_VEC = dn.compute_earth_to_sun()


## Spacecraft Initial States

In [ ]:
# Initial condition of reference orbit
h = 600  # km  height for perigee
e = 0.001
a = (R_EARTH + h) / (1 - e)
i = 0.6
W = 0
w = 0
E = 0

# satellite initial condition
# dr = np.array([1.2e-5, 1.5e-5, 1.3e-5])   # relative position vector to ref
# dv = np.array([1.1e-5, 1.4e-5, 1.7e-5])  # relative velocity to ref

dr = np.array([0.1, 0, 0])   # relative position vector to ref
dv = np.array([1e-5, 0, 0])  # relative velocity to ref

# orbital period
T = 2 * jnp.pi * jnp.sqrt(a**3/MU_EARTH_KM)

# compute orbit elements
oe_chief = np.array([a, e, i, W, w, E])
r0_chief, v0_chief = pykep.core.par2ic(oe_chief, MU_EARTH_KM)
r0_sat, v0_sat = r0_chief + dr, v0_chief + dv

# convert to JAX array
x0_chief_eci = jnp.array(np.hstack([r0_chief, v0_chief]))

# Initial  States
s0_rel_chief = jnp.hstack([dn.eci_to_lvlh(x0_chief_eci, x0_chief_eci)])
print(f"Should be uniformly zero = {s0_rel_chief}")

## Time Propagation Settings

In [ ]:
# Propagation time settings
dt = 0.1
tspan_quarter_periods = np.arange(0, 0.1 * T, dt)   # propagation step
print(f"Tspan size: {tspan_quarter_periods.size}")

## JIT Dynamics Functions

In [ ]:
# JIT the dynamics functions
t = 0  # we can assume this because there is no change in the parameters
fd_absolute = jax.jit(ct.discretize(ct.absolute_dynamics, dt))

x_chief_eci_traj = ct.propagate_absolute_from_control(x0_chief_eci, ct.zero_control, tspan_quarter_periods, fd_absolute)

interp_abs_chief_jit = jax.jit(ct.interp_fixed(tspan_quarter_periods, x_chief_eci_traj[:, :-1]))
fd_relative_4T = jax.jit(ct.discretize(ct.relative_dynamics, dt, params=(GAMMA_SRP, GAMMA_DRAG, PSI), state_eci_ref_func=interp_abs_chief_jit))
fd_relative_4T_back = jax.jit(ct.discretize(ct.relative_dynamics, -dt, params=(GAMMA_SRP, GAMMA_DRAG, PSI), state_eci_ref_func=interp_abs_chief_jit))

# NN Propagator
fd_relative_4T_NN = ct.discretize_Euler(ct.relative_dynamics_neural_net, dt,
                                        params=np.array([GAMMA_SRP, GAMMA_DRAG, PSI]),
                                        state_eci_ref_func=interp_abs_chief_jit,
                                        model=model,
                                        min_max_in=norm_inputs, min_max_out=norm_outputs)

## Generate Initial Condition by back propagation

In [ ]:
Nctrl = 10  # control steps
control_interval = 10   # apply same ctrl for this step
Nsteps = Nctrl * control_interval

if dt * Nctrl >= tspan_quarter_periods[-1]:
    print("Final Epoch too large!")

t_final = dt * Nsteps

# define control bounds
min_T = -250e-3  # N
max_T = 250e-3 # N

u_min = min_T/MASS_SAT  # m / s
u_max = max_T/MASS_SAT  # m / s

# propagate the state backwards
u_random = ct.rand_box_control(np.ones(Nctrl), u_min, u_max).transpose()  # T x 3
u_mat_rep = np.repeat(u_random, control_interval, axis=0)   # (intervalxT) x 3
u_random_rep = u_mat_rep.flatten()

state_init = np.zeros(6)
states_feasible = np.zeros((6, Nsteps+1))

states_feasible[:,-1] = state_init

for i in range(Nsteps):
    current_t = dt * (Nsteps-i)
    state_init = fd_relative_4T_back(state_init, current_t, u_mat_rep[Nsteps-1-i, :])
    states_feasible[:, Nsteps-1-i] = state_init

print("Initial Condition:", state_init)
ct.plot_control_trajectory(states_feasible)

In [ ]:
# Forward Propagate the trajectory for check
t_prop = np.arange(0, t_final, dt)
s_forward = ct.propagate_relative_from_control(state_init, u_random_rep, t_prop,  fd_relative_4T)
ct.plot_control_trajectory(s_forward)

## Test NN Outputs

In [ ]:
acc_NN = ct.relative_dynamics_neural_net(state_init, 0, np.zeros(3),
                                  params=np.array([GAMMA_SRP, GAMMA_DRAG, PSI]),
                                  state_eci_ref_func=interp_abs_chief_jit,
                                  model=model,
                                  min_max_in=norm_inputs, min_max_out=norm_outputs)
print(acc_NN)

## Test NN discrete dynamics propagator

In [ ]:
# Propate with NN
t_prop = np.arange(0, t_final, dt)
s_forward_NN = ct.propagate_relative_from_control(state_init, u_random_rep, t_prop,  fd_relative_4T_NN)
ct.plot_control_trajectory2(s_forward, s_forward_NN)

In [ ]:
modify_initial = True

if modify_initial:
    dr = np.array([1.2e-5, 1.5e-5, 1.3e-5])   # relative position vector to ref
    dv = np.array([1.1e-5, 1.4e-5, 1.7e-5])  # relative velocity to ref
    r0_sat, v0_sat = r0_chief + dr, v0_chief + dv
    x0_sat_eci = jnp.array(np.hstack([r0_sat, v0_sat]))
    state_init = jnp.hstack([dn.eci_to_lvlh(x0_chief_eci, x0_sat_eci)])

## Test Control Optimization with Scipy Optimize

In [ ]:
# Test optimizing with scipy minimize
only_mpc = True
final_is_hard_constraint = False   # set terminal constraint as cost or hard constraint

if not only_mpc:
    if final_is_hard_constraint:
        ## Solve with final point as hard constraint
        s_optimal, u_optimal, tspan_optimize, u_initial, cost, constraint_vl \
          = ct.shooting_with_scipy(Nctrl, control_interval,
                                   dt, u_min, u_max, state_init, fd_relative_4T,
                                   u_guess = u_random.flatten(),
                                   maxiter=5000)
    else:
        s_optimal, u_optimal, tspan_optimize, u_initial \
          = ct.optimize_with_scipy(Nctrl, control_interval,
                                   dt, u_min, u_max, state_init, fd_relative_4T,
                                   u_guess= None, #u_random.flatten(),
                                   use_MBH = False,
                                   maxiter=5000, stage_w=0, terminal_w=1e10)

    print("Finished!")

## Plot control results
- plot the outputs from the control sequence

In [ ]:
# Plot Result
if not only_mpc:
    print(u_optimal.reshape(-1, 3).shape)
    print(tspan_optimize.shape)

    u_optimal_mat = u_optimal.reshape(-1, 3)
    s_init = ct.propagate_relative_from_control(state_init, u_initial, tspan_optimize, fd_relative_4T)

    plt.plot(tspan_optimize, np.linalg.norm(u_initial.reshape(-1, 3), axis=1), label="u initial")
    plt.plot(tspan_optimize, np.linalg.norm(u_optimal_mat, axis=1), label="u optimal")
    plt.plot(tspan_optimize, np.linalg.norm(u_mat_rep, axis=1), label="u feasible")
    plt.xlabel("Time (s)")
    plt.ylabel("$||u||_2$")
    plt.legend()
    plt.show()

    dn.plot_relative_orbit(tspan_optimize, s_optimal[:, :-1].T)

In [ ]:
if not only_mpc:
    ct.plot_control_trajectory2(s_optimal, s_init)

## Try MPC with Scipy Optimize

In [ ]:
## Test MPC
P_outer = 1e10 * np.eye(6)                    # terminal state cost matrix
Q_outer = 1e7 * np.eye(6)                    # state cost matrix
rho_outer = 1.                               # trust region parameter
u_max_outer = u_max                  # control effort bound
tol = 5e-1                           # convergence tolerance
max_iters = 10                       # maximum number of scipy optimize
u_dim_outer = 3

N_total_mpc = Nsteps
t_last_mpc = N_total_mpc * dt
N_horizon_mpc = 10

t_mpc, s_mpc, u_mpc = ct.run_MPC_with_scipy_NN(state_init, t_last_mpc, N_horizon_mpc,
                                                P_outer, Q_outer, u_max, rho_outer,
                                                tol, max_iters, u_dim_outer, dt, fd_relative_4T,
                                                fd_relative_4T_NN)


In [ ]:
## Plot MPC Result
ct.plot_mpc_control(t_mpc, s_mpc, u_mpc, u_max)

In [ ]:
ct.plot_mpc_trajectory(s_mpc, N_total_mpc, N_horizon_mpc)